In [8]:
import numpy as np
import pandas as pd
import mediapipe as mp
import os
import cv2
import joblib



In [9]:
mp_holistic = mp.solutions.holistic #Holistic model 
mp_drawing = mp.solutions.drawing_utils #drawing utiliites
model_filename = 'savedModels/random_forest_model.joblib'
model = joblib.load(model_filename)

In [10]:
def draw_styled_landmarks(image, results):
    
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                             mp_drawing.DrawingSpec(color=(80,110,10),thickness = 1, circle_radius=1),
    #                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
    #                          )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10),thickness = 2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76),thickness = 2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,120,60),thickness = 2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,56,221),thickness=2,circle_radius=2))


In [11]:
def extract_keypoints(results):
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)

    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    pose = pose[:22]
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])


In [12]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [13]:
cap = cv2.VideoCapture(-1)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        cv2.putText(image, 'Press Q to exit', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 50), 2, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
        keypoints = extract_keypoints(results)
        if keypoints.any():
            prediction = model.predict([keypoints])
            cv2.putText(image, 'Prediction: {}'.format(prediction[0]), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1728095636.130785  113858 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1728095636.131865  116392 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: RENOIR (renoir, LLVM 15.0.7, DRM 3.57, 6.8.0-45-generic)
W0000 00:00:1728095636.194303  116384 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728095636.214810  116387 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728095636.216021  116384 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728095636.216310  116387 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling suppo